# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [17]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [18]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [19]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

***Note*** in our implementaiton true **batch_size** is always 2 fold as we agument our data. if if batch size is 10, true batch size at the run time is 20, as twice the number of data set loaded are added to the batch


## Generator
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [20]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [21]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [22]:
# Helper function to normalise the data
def normalize_data(data):
    return data/127.5-1

In [23]:
# Helper function to initialize all the batch image data and labels
def init_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [24]:
def load_batch_images(source_path, folder_list, batch_num, batch_size, t):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    # We will also build a agumented batch data
    batch_data_aug,batch_labels_aug = init_batch_data(batch_size)

    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            batch_data[folder,idx] = resized
            batch_data_aug[folder,idx] = cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1]))      

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
    
    batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
    batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
    return batch_data, batch_labels

In [25]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield load_batch_images(source_path, folder_list, batch, batch_size, t) 
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield load_batch_images(source_path, folder_list, num_batches, batch_size, t)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [26]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model #1

In [27]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [28]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [29]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [30]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [31]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [33]:
batch_size = 10
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
67/67 [==============================] - 12s 174ms/step - loss: 0.6854 - categorical_accuracy: 0.7164 - val_loss: 0.7937 - val_categorical_accuracy: 0.7200

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.68538-0.71642-0.79369-0.72000.h5
Epoch 2/10
67/67 [==============================] - 11s 161ms/step - loss: 0.4354 - categorical_accuracy: 0.8532 - val_loss: 0.6797 - val_categorical_accuracy: 0.7150

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.43536-0.85323-0.67969-0.71500.h5
Epoch 3/10
67/67 [==============================] - 10s 149ms/step - loss: 0.5200 - categorical_accuracy: 0.7836 - val_loss: 0.9677 - val_categorical_accuracy: 0.6350

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.52000-0.78358-0.96765-0.63500.h5
Epoch 4/10
67/67 [==============================] - 10s 154ms/step - loss: 0.5662 - categorical_accuracy: 0.7861 - val_loss: 0.7605 - val_categorical_accurac

In [34]:
batch_size = 20
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
67/67 [==============================] - 13s 190ms/step - loss: 0.1951 - categorical_accuracy: 0.9353 - val_loss: 0.7364 - val_categorical_accuracy: 0.8100

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.19512-0.93532-0.73644-0.81000.h5
Epoch 2/20
67/67 [==============================] - 11s 169ms/step - loss: 0.2021 - categorical_accuracy: 0.9279 - val_loss: 0.5978 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.20211-0.92786-0.59782-0.76000.h5
Epoch 3/20
67/67 [==============================] - 11s 160ms/step - loss: 0.2033 - categorical_accuracy: 0.9328 - val_loss: 0.7070 - val_categorical_accuracy: 0.7850

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.20330-0.93284-0.70703-0.78500.h5
Epoch 4/20
67/67 [==============================] - 11s 171ms/step - loss: 0.2000 - categorical_accuracy: 0.9328 - val_loss: 0.7221 - val_categorical_accurac

In [35]:
batch_size = 30
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
67/67 [==============================] - 11s 169ms/step - loss: 0.2160 - categorical_accuracy: 0.9154 - val_loss: 0.6446 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.21598-0.91542-0.64455-0.80000.h5
Epoch 2/30
67/67 [==============================] - 10s 151ms/step - loss: 0.1760 - categorical_accuracy: 0.9378 - val_loss: 0.6708 - val_categorical_accuracy: 0.8100

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.17596-0.93781-0.67076-0.81000.h5
Epoch 3/30
67/67 [==============================] - 11s 165ms/step - loss: 0.1533 - categorical_accuracy: 0.9453 - val_loss: 0.6157 - val_categorical_accuracy: 0.8100

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.15333-0.94527-0.61573-0.81000.h5
Epoch 4/30
67/67 [==============================] - 10s 156ms/step - loss: 0.1685 - categorical_accuracy: 0.9527 - val_loss: 0.6680 - val_categorical_accurac

67/67 [==============================] - 11s 170ms/step - loss: 0.1658 - categorical_accuracy: 0.9403 - val_loss: 0.6700 - val_categorical_accuracy: 0.8000

Epoch 00027: saving model to model_init_2018-10-2019_33_03.656771/model-00027-0.16577-0.94030-0.66999-0.80000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 5.960464760645934e-11.
Epoch 28/30
67/67 [==============================] - 10s 154ms/step - loss: 0.1536 - categorical_accuracy: 0.9428 - val_loss: 0.7099 - val_categorical_accuracy: 0.7800

Epoch 00028: saving model to model_init_2018-10-2019_33_03.656771/model-00028-0.15363-0.94279-0.70987-0.78000.h5
Epoch 29/30
67/67 [==============================] - 11s 168ms/step - loss: 0.2296 - categorical_accuracy: 0.9055 - val_loss: 0.6255 - val_categorical_accuracy: 0.8200

Epoch 00029: saving model to model_init_2018-10-2019_33_03.656771/model-00029-0.22960-0.90547-0.62550-0.82000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 2.980232380322967e-11.
Epoc

In [36]:
batch_size = 50
num_epochs = 50
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
67/67 [==============================] - 12s 177ms/step - loss: 0.2242 - categorical_accuracy: 0.9204 - val_loss: 0.7112 - val_categorical_accuracy: 0.7850

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.22424-0.92040-0.71121-0.78500.h5
Epoch 2/50
67/67 [==============================] - 10s 154ms/step - loss: 0.1548 - categorical_accuracy: 0.9428 - val_loss: 0.5361 - val_categorical_accuracy: 0.8200

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.15483-0.94279-0.53608-0.82000.h5
Epoch 3/50
67/67 [==============================] - 11s 160ms/step - loss: 0.1813 - categorical_accuracy: 0.9403 - val_loss: 0.7708 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.18125-0.94030-0.77081-0.80000.h5
Epoch 4/50
67/67 [==============================] - 11s 168ms/step - loss: 0.1693 - categorical_accuracy: 0.9353 - val_loss: 0.5825 - val_categorical_accurac

67/67 [==============================] - 11s 160ms/step - loss: 0.1018 - categorical_accuracy: 0.9701 - val_loss: 0.6541 - val_categorical_accuracy: 0.8400

Epoch 00027: saving model to model_init_2018-10-2019_33_03.656771/model-00027-0.10179-0.97015-0.65410-0.84000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 7.275957959772868e-15.
Epoch 28/50
67/67 [==============================] - 10s 154ms/step - loss: 0.1851 - categorical_accuracy: 0.9378 - val_loss: 0.6118 - val_categorical_accuracy: 0.7900

Epoch 00028: saving model to model_init_2018-10-2019_33_03.656771/model-00028-0.18507-0.93781-0.61176-0.79000.h5
Epoch 29/50
67/67 [==============================] - 11s 162ms/step - loss: 0.2158 - categorical_accuracy: 0.9030 - val_loss: 0.8173 - val_categorical_accuracy: 0.7850

Epoch 00029: saving model to model_init_2018-10-2019_33_03.656771/model-00029-0.21582-0.90299-0.81734-0.78500.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 3.637978979886434e-15.
Epoc

In [37]:
batch_size = 90
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
67/67 [==============================] - 12s 186ms/step - loss: 0.1986 - categorical_accuracy: 0.9254 - val_loss: 0.5849 - val_categorical_accuracy: 0.8050

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.19858-0.92537-0.58493-0.80500.h5
Epoch 2/30
67/67 [==============================] - 11s 165ms/step - loss: 0.1880 - categorical_accuracy: 0.9378 - val_loss: 0.8064 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.18798-0.93781-0.80636-0.76000.h5
Epoch 3/30
67/67 [==============================] - 11s 170ms/step - loss: 0.1582 - categorical_accuracy: 0.9453 - val_loss: 0.6511 - val_categorical_accuracy: 0.8050

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.15817-0.94527-0.65105-0.80500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.776356923772673e-18.
Epoch 4/30
67/67 [==============================] - 11s 163ms/step - loss: 0

67/67 [==============================] - 11s 157ms/step - loss: 0.2162 - categorical_accuracy: 0.9303 - val_loss: 0.8152 - val_categorical_accuracy: 0.7500

Epoch 00027: saving model to model_init_2018-10-2019_33_03.656771/model-00027-0.21619-0.93035-0.81515-0.75000.h5
Epoch 28/30
67/67 [==============================] - 11s 169ms/step - loss: 0.1792 - categorical_accuracy: 0.9328 - val_loss: 0.4960 - val_categorical_accuracy: 0.8400

Epoch 00028: saving model to model_init_2018-10-2019_33_03.656771/model-00028-0.17916-0.93284-0.49601-0.84000.h5
Epoch 29/30
67/67 [==============================] - 11s 165ms/step - loss: 0.1458 - categorical_accuracy: 0.9627 - val_loss: 0.7743 - val_categorical_accuracy: 0.7950

Epoch 00029: saving model to model_init_2018-10-2019_33_03.656771/model-00029-0.14575-0.96269-0.77425-0.79500.h5
Epoch 30/30
67/67 [==============================] - 11s 160ms/step - loss: 0.1604 - categorical_accuracy: 0.9502 - val_loss: 0.5820 - val_categorical_accuracy: 0.820

In [38]:
batch_size = 90
num_epochs = 10

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
8/8 [==============================] - 2s 208ms/step - loss: 0.2286 - categorical_accuracy: 0.8750 - val_loss: 0.3565 - val_categorical_accuracy: 0.8750

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.22857-0.87500-0.35645-0.87500.h5
Epoch 2/10
8/8 [==============================] - 1s 161ms/step - loss: 0.1333 - categorical_accuracy: 0.9375 - val_loss: 0.4650 - val_categorical_accuracy: 0.9000

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.13329-0.93750-0.46503-0.90000.h5
Epoch 3/10
8/8 [==============================] - 1s 153ms/step - loss: 0.2545 - categorical_accuracy: 0.9375 - val_loss: 0.8256 - val_categorical_accuracy: 0.6500

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.25448-0.93750-0.82564-0.65000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.1684044479646887e-22.
Epoch 4/10
8/8 [==============================] - 1s 183ms/step - loss: 0.1826 - cat

In [40]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 2s 196ms/step - loss: 0.3110 - categorical_accuracy: 0.8958 - val_loss: 0.4754 - val_categorical_accuracy: 0.8500

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.31096-0.89583-0.47540-0.85000.h5
Epoch 2/20
8/8 [==============================] - 1s 148ms/step - loss: 0.2198 - categorical_accuracy: 0.9375 - val_loss: 0.8738 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.21977-0.93750-0.87380-0.80000.h5
Epoch 3/20
8/8 [==============================] - 1s 162ms/step - loss: 0.0902 - categorical_accuracy: 1.0000 - val_loss: 1.0319 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.09018-1.00000-1.03191-0.75000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 8.470329874862065e-25.
Epoch 4/20
8/8 [==============================] - 1s 183ms/step - loss: 0.2552 - cate

In [42]:
batch_size = 90
num_epochs = 10

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
8/8 [==============================] - 2s 199ms/step - loss: 0.1214 - categorical_accuracy: 0.9583 - val_loss: 0.2893 - val_categorical_accuracy: 0.9000

Epoch 00001: saving model to model_init_2018-10-2019_33_03.656771/model-00001-0.12139-0.95833-0.28930-0.90000.h5
Epoch 2/10
8/8 [==============================] - 1s 182ms/step - loss: 0.0803 - categorical_accuracy: 0.9792 - val_loss: 0.4775 - val_categorical_accuracy: 0.8500

Epoch 00002: saving model to model_init_2018-10-2019_33_03.656771/model-00002-0.08029-0.97917-0.47746-0.85000.h5
Epoch 3/10
8/8 [==============================] - 1s 155ms/step - loss: 0.2085 - categorical_accuracy: 0.8750 - val_loss: 0.5700 - val_categorical_accuracy: 0.8250

Epoch 00003: saving model to model_init_2018-10-2019_33_03.656771/model-00003-0.20850-0.87500-0.56999-0.82500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 2.5849395370062454e-29.
Epoch 4/10
8/8 [==============================] - 1s 161ms/step - loss: 0.2383 - cat